In [1]:
import torch
import torch.nn as nn
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
# img_size, patch_size, num_hiddens, batch_size = 96, 16, 512, 4
# x = torch.ones(batch_size, 3, img_size, img_size)
# conv_layer = nn.LazyConv2d(num_hiddens, kernel_size=(patch_size, patch_size), stride=(patch_size, patch_size))

In [3]:
# conv_layer(x).flatten(start_dim=2).shape

In [56]:
x = torch.arange(16).reshape(4, 4).type(torch.float32)
x

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.]])

In [64]:
# 展开维度、kernel size、stride
a = x.unfold(0, 2, 2)
a

tensor([[[ 0.,  4.],
         [ 1.,  5.],
         [ 2.,  6.],
         [ 3.,  7.]],

        [[ 8., 12.],
         [ 9., 13.],
         [10., 14.],
         [11., 15.]]])

In [79]:
b

tensor([[[[ 0.,  1.],
          [ 4.,  5.]],

         [[ 2.,  3.],
          [ 6.,  7.]]],


        [[[ 8.,  9.],
          [12., 13.]],

         [[10., 11.],
          [14., 15.]]]])

In [80]:
b = a.unfold(1, 2, 2)
b.reshape(4, 2, 2)

tensor([[[ 0.,  1.],
         [ 4.,  5.]],

        [[ 2.,  3.],
         [ 6.,  7.]],

        [[ 8.,  9.],
         [12., 13.]],

        [[10., 11.],
         [14., 15.]]])

In [85]:
def pad_to_divide(img_batch, kernel_size):
    # img_batch shape:(batch_size, n_channels, h, w)
    h, w = img_batch.shape[2:]
    k_h, k_w = kernel_size
    pad_num_h, pad_num_w = 0 if h % k_h == 0 else k_h - h % k_h, 0 if w % k_w == 0 else k_w - w % k_w
    pad_num_u = pad_num_h // 2
    pad_num_d = pad_num_h - pad_num_u
    pad_num_l = pad_num_w // 2
    pad_num_r = pad_num_w - pad_num_l
    return nn.functional.pad(img_batch, pad=(pad_num_l, pad_num_r, pad_num_u, pad_num_d))

In [88]:
def patchify(img_batch, patch_size):
    img_batch = pad_to_divide(img_batch, patch_size)
    p_h, p_w = patch_size
    # img_batch shape:(batch_size, num_channels, h, w)
    batch_size, num_chnls, h, w = img_batch.shape
    return img_batch.unfold(2, patch_size[0], patch_size[0]).unfold(3, patch_size[1], patch_size[1]).reshape(batch_size, num_chnls, -1,p_h, p_w).transpose(1,2)

In [91]:
x

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.]])

In [92]:
output = patchify(x.unsqueeze(0).unsqueeze(0), (2,2))
output.shape

torch.Size([1, 4, 1, 2, 2])